# part4:向量数据库实习
我们使用的数据库是PostgreSQL 数据库，我们利用python连接到数据库，执行SQL语句。

In [2]:
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import psycopg2
import csv
# 连接到 PostgreSQL 数据库
def connect2PG():
    conn = psycopg2.connect(
      user="postgres",
      password="ren314159",
      host="127.0.0.1",
      port=5433, 
      database="postgres"
    )
    return conn

# 用于执行 SQL 语句
def execSQL(conn, query, params = None):
    with conn.cursor() as cur:
        if params:
            cur.execute(query, params)
        else:
            cur.execute(query)
        conn.commit()
   
# 用来返回sql语句的查询结果
def querySQL(conn, query, params = None):
    with conn.cursor() as cur:
        # 执行 SQL 查询
        if params:
            cur.execute(query, params)
        else:
            cur.execute(query)
        # 获取查询结果
        result = cur.fetchall()
        # 返回结果
        return result
    
# SQL初始化，没有表则创建表
def SQL_initial():
    sql_table1 =  '''
    CREATE TABLE IF NOT EXISTS yttlj(
      phaseId INT PRIMARY KEY,
      phaseText TEXT
    );
    '''
    sql_table2 =  '''
    CREATE TABLE IF NOT EXISTS person(
      personId INT PRIMARY KEY,
      personName TEXT
    );
    '''
    sql_table3 =  '''
    CREATE TABLE IF NOT EXISTS coCurrence(
      personId1 INT PRIMARY KEY,
      personId2 INT,
      counts INT
    );
    '''
    conn = connect2PG()
    execSQL(conn,sql_table1)
    execSQL(conn,sql_table2)
    execSQL(conn,sql_table3)

## 创建小说表
将小说表的内容插入到数据表中

In [9]:
SQL_initial()
with open("yttlj.csv", 'r') as f:
    reader = csv.reader(f)
    next(reader)  # 跳过表头
    conn = connect2PG()
    sql = "INSERT INTO yttlj(phaseId, phaseText) VALUES (%s, %s)"
    for row in reader:
        execSQL(conn, sql, row)

## 创建人物表
将人物表的内容插入到数据表中

In [10]:
# 创建人物数据
personName_list = ['张三丰', '张翠山', '殷素素', '俞岱岩', '俞莲舟', '殷梨亭', '纪晓芙', '张无忌', '周芷若', '赵敏', '殷离', '小昭', '黛绮丝', '谢逊', '朱长龄', '朱九真', '杨逍', '范遥', '灭绝师太', '何足道', '胡青牛']
conn = connect2PG()
sql = "INSERT INTO person(personId, personName) VALUES (%s, %s)"
for i in range(len(personName_list)):
    params = (i, personName_list[i])
    execSQL(conn, sql, params)

## 建立共现表
构建思路，对于人物表中的每一个人物，创建一个长度为4377的位置向量，向量第i个位置来表示该人物是否在第i段文本中出现，出现则为1，未出现则为0。那么两个人物之间的共现次数则为两人物对应的位置向量的数量积。

In [52]:
all_sql = '''
-- 创建位置向量
CREATE TABLE IF NOT EXISTS person_position_vector (
    personId INT,
    vector VECTOR(4377)  -- 使用pgvector来存储位置向量
);

-- 创建独一索引，便于更新vector
CREATE UNIQUE INDEX IF NOT EXISTS idx_person_id ON person_position_vector (personId);

INSERT INTO person_position_vector (personId, vector)
SELECT
    p.personId,
    ARRAY(
        SELECT
            CASE
                WHEN y.phaseText LIKE '%' || p.personName || '%' THEN 1
                ELSE 0
            END
        FROM
            yttlj y
    ) AS vector
FROM
    person p
ON CONFLICT (personId) DO UPDATE
SET
    vector = EXCLUDED.vector;

    
DROP TABLE IF EXISTS coCurrence;
CREATE TABLE coCurrence (
    personId1 INT NOT NULL,
    personId2 INT NOT NULL,
    counts FLOAT DEFAULT 0,
    PRIMARY KEY (personId1, personId2)
);

-- 计算共现次数
INSERT INTO coCurrence (personId1, personId2, counts)
SELECT
    p1.personId AS personId1,
    p2.personId AS personId2,
    - (p1v.vector <#> p2v.vector) AS counts
FROM
    person p1,
    person p2,
    person_position_vector p1v,
    person_position_vector p2v
WHERE
    p1.personId <= p2.personId  -- 避免重复计算
    AND p1v.personId = p1.personId
    AND p2v.personId = p2.personId
ON CONFLICT (personId1, personId2)
DO UPDATE SET counts = coCurrence.counts + EXCLUDED.counts;

'''
conn = connect2PG()
execSQL(conn, all_sql)

## 通过矩阵分解得到词向量表示
首先提取出共现矩阵，然后对矩阵进行分解，我们采取的分解方式为奇异值分解。

In [3]:
conn = connect2PG()
sql = "SELECT personId1, personId2, counts FROM coCurrence;"
coCurrence_data = querySQL(conn, sql)
coCurrence_df = pd.DataFrame(coCurrence_data, columns=['personId1', 'personId2', 'counts'])
# 获取所有人物ID
person_ids = sorted(coCurrence_df['personId1'].unique())
# 构建共现矩阵
coCurrence_matrix = pd.DataFrame(0, index=person_ids, columns=person_ids)

for _, row in coCurrence_df.iterrows():
    if row['personId1'] != row['personId2']:
      coCurrence_matrix.loc[row['personId1'], row['personId2']] = row['counts']
      coCurrence_matrix.loc[row['personId2'], row['personId1']] = row['counts']  
    else:
      coCurrence_matrix.loc[row['personId1'], row['personId2']] = 0
   
# 使用TruncatedSVD进行SVD分解
svd = TruncatedSVD(n_components=2)  # n_components是你想要的特征数量
features = svd.fit_transform(coCurrence_matrix)


## 将词向量存入pgvector，计算相似度
将词向量存入pgvector，按照余弦相似度计算最相似的人物。

In [ ]:
# 创建person_vector表来存词向量
conn = connect2PG()
sql_vector = '''
CREATE TABLE IF NOT EXISTS person_vector (
    personId INT PRIMARY KEY,
    vector VECTOR(2)  -- 根据需要调整维度
);
'''
execSQL(conn, sql_vector)

# 插入词向量
sql_insert = "INSERT INTO person_vector (personId, vector) VALUES (%s, %s)"
for person_id, vector in zip(coCurrence_matrix.index, features):
    vector_list = list(vector)
    params = (person_id, vector_list)
    execSQL(conn, sql_insert, params)

# 查询最相似的人物
sql_search_similarity = """
WITH RankedSimilarity AS (
    SELECT 
        p1.personId AS person_id, 
        p1.personName AS person_name,
        p2.personId AS related_person_id, 
        p2.personName AS related_person_name,
        1 - (pv1.vector <=> pv2.vector) AS cosine_similarity,
        ROW_NUMBER() OVER (PARTITION BY p1.personId ORDER BY 1 - (pv1.vector <=> pv2.vector) DESC) AS rank
    FROM 
        person p1
    JOIN 
        person_vector pv1 ON p1.personId = pv1.personId
    JOIN 
        person_vector pv2 ON pv1.personId != pv2.personId
    JOIN 
        person p2 ON pv2.personId = p2.personId
)
SELECT 
    person_id, 
    person_name, 
    related_person_id, 
    related_person_name, 
    cosine_similarity
FROM 
    RankedSimilarity
WHERE 
    rank = 1;
"""
most_similarity = querySQL(conn, sql_search_similarity)
print(most_similarity)
